In [1]:
import numpy as np
import pandas as pd
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('ECG200_TRAIN.csv', header=None, delimiter='  ')
df

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,-1.0,0.502055,0.542163,0.722383,1.428885,2.136516,2.281149,1.936274,1.468890,1.008845,...,0.931043,0.610298,0.638894,0.684679,0.583238,0.640522,0.708585,0.705011,0.713815,0.433765
1,1.0,0.147647,0.804668,0.367771,0.243894,0.026614,-0.274402,0.096731,-0.747731,-1.609777,...,-0.533503,-0.400228,0.176084,1.111768,2.438428,2.734889,1.736054,0.036857,-1.265074,-0.208024
2,-1.0,0.316646,0.243199,0.370471,1.063738,1.678187,1.759558,1.697717,1.612159,1.168188,...,0.764229,0.610621,0.552900,0.566786,0.604002,0.777068,0.812345,0.748848,0.818042,0.539347
3,-1.0,1.168874,2.075901,1.760141,1.606446,1.949046,1.302842,0.459332,0.516412,0.852180,...,0.419006,0.723888,1.323947,2.136488,1.746597,1.470220,1.893512,1.256949,0.800407,0.731540
4,1.0,0.648658,0.752026,2.636231,3.455716,2.118157,0.520620,-0.188627,0.780818,0.933775,...,-0.097869,-0.136787,-0.340237,-0.089441,-0.080297,-0.192584,-0.304704,-0.454556,0.314590,0.582190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,0.581277,0.876188,1.042767,1.796120,2.541399,2.246653,1.500387,1.031521,0.382672,...,1.002770,0.907869,0.916457,0.923975,0.767357,0.656223,0.762357,0.501373,-0.333336,-0.524546
96,-1.0,2.689017,2.708703,2.008381,2.235800,1.516982,0.029916,-0.561346,-0.793702,-0.979371,...,-0.136610,-0.072176,-0.082738,-0.138468,-0.120396,-0.089411,-0.243141,-0.119710,0.124042,0.273463
97,-1.0,0.197677,0.455417,0.973110,1.935956,2.259463,1.741341,1.158296,0.418241,-0.071605,...,0.482452,0.325569,0.247991,0.184127,0.050358,0.241988,0.331451,-0.120006,0.042423,0.343293
98,1.0,0.179500,1.038409,1.946421,2.705141,1.670706,-0.101167,-1.578876,-0.750906,0.175310,...,0.324323,0.330489,0.111953,0.448948,0.567132,0.136757,0.444768,0.151050,0.193378,0.451709


In [3]:
test_ratio = 0.3
train_ratio = 0.7
pos = df[df[0]==1]
neg = df[df[0]==-1]

**K-Folds Verification (k=3)**

In [4]:
k = 3
df1 = []
df2 = []

for i in range(k):
    df1.append(pd.DataFrame(np.array_split(pos, k)[i]))
    df2.append(pd.DataFrame(np.array_split(neg, k)[i]))

In [5]:
totacc = []
totf1 = []
for i in range(k):
    
    test = df1[i].append(df2[i], ignore_index = True)    
    train = pd.DataFrame(columns = df.columns)
    
    for j in range(k):
        
        if i == j:
            continue
            
        train = train.append(df1[j], ignore_index=True)
        train = train.append(df2[j], ignore_index=True)
        
    X_train = train.drop([0], axis=1)
    y_train = train[0]
    X_test = test.drop([0], axis=1)
    y_test = test[0]
    
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    pred = knn.predict(X_test)
    
    acc = '{:.2f}'.format(accuracy_score(y_test,pred))
    f1 = '{:.2f}'.format(f1_score(y_test, pred, average='macro'))
    
    print ("Fold {} \n\tAccuracy = {}, F1 Score = {}".format(i+1, acc, f1))
    totacc.append(float(acc))
    totf1.append(float(f1))

print ('\nMean Accuracy =', '{:.2f}'.format(np.mean(totacc)))
print ('Mean F1 Score =', '{:.2f}'.format(np.mean(totf1)))

Fold 1 
	Accuracy = 0.76, F1 Score = 0.73
Fold 2 
	Accuracy = 0.82, F1 Score = 0.78
Fold 3 
	Accuracy = 0.85, F1 Score = 0.82

Mean Accuracy = 0.81
Mean F1 Score = 0.78


**Evaluation with Sklearn**

In [6]:
X = df.drop(0, axis=1)
y = df[0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
    
acc = '{:.2f}'.format(accuracy_score(y_test,pred))
f1 = '{:.2f}'.format(f1_score(y_test, pred, average='macro'))
    
print ('Accuracy : ', acc)  
print('F1 Score : ', f1)

Accuracy :  0.83
F1 Score :  0.81


**Comparing**

By using F-1 score, K-fold yields 0.78 whereas Sklean yields 0.81